In [15]:
import numpy as np
import random
import nltk
import tensorflow
nltk.download('wordnet')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import json
import pickle
import warnings
warnings.filterwarnings("ignore")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
intents= json.loads(open('intents.json').read())
print("Processing intents")

Processing intents


In [17]:
nltk.download('punkt')
words = []
classes = []
documents = []
ignore_words = ['?','!',',','.']
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w= nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))
    if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(classes)
print(documents)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['greetings', 'goodbye', 'upload', 'access', 'download', 'watch', 'update', 'badges']
[(['hey'], 'greetings'), (['hello'], 'greetings'), (['hi', 'there'], 'greetings'), (['hi'], 'greetings'), (['haii'], 'greetings'), (['hai'], 'greetings'), (['Greetings'], 'greetings'), ([], 'greetings'), (['what', "'s", 'up', '?'], 'greetings'), (['c'], 'goodbye'), (['bye'], 'goodbye'), (['goodbye'], 'goodbye'), (['Thank', 'you', 'so', 'much', '.'], 'goodbye'), (['good', 'day'], 'goodbye'), (['farewell'], 'goodbye'), (['adieu'], 'goodbye'), (['bye'], 'goodbye'), (['thankyou'], 'goodbye'), (['thanku'], 'goodbye'), (['Ca', "n't", 'upload', 'home', 'work'], 'upload'), (['I', "'m", 'unable', 'to', 'upload', 'my', 'homework'], 'upload'), (['Not', 'able', 'to', 'submit', 'home', 'work'], 'upload'), (['I', "'m", 'unable', 'to', 'turn', 'in', 'my', 'homework'], 'upload'), (['I', 'ca', "n't", 'upload'

In [18]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words] 
words= sorted(list(set(words)))
classes = sorted(list(set(classes)))
print(classes)


['access', 'badges', 'download', 'goodbye', 'greetings', 'update', 'upload', 'watch']


In [19]:
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "Unique stemmed words", words)

195 documents
8 classes ['access', 'badges', 'download', 'goodbye', 'greetings', 'update', 'upload', 'watch']
178 Unique stemmed words ["'d", "'m", "'re", "'s", "'ve", 'a', 'abl', 'access', 'adieu', 'allow', 'am', 'an', 'any', 'ap', 'apply', 'ar', 'assign', 'avail', 'bad', 'badg', 'batch', 'be', 'being', 'best', 'but', 'bye', 'c', 'ca', 'can', 'cap', 'challeng', 'class', 'collect', 'comput', 'connect', 'could', 'dat', 'day', 'did', 'difficul', 'difficult', 'display', 'do', 'doe', 'download', 'earn', 'engl', 'ent', 'er', 'fail', 'farewel', 'feat', 'fil', 'for', 'from', 'gath', 'get', 'good', 'goodby', 'greet', 'had', 'hai', 'hav', 'hello', 'hey', 'hi', 'hom', 'homework', 'how', 'hw', 'i', 'import', 'imposs', 'in', 'ind', 'instal', 'into', 'iphon', 'is', 'issu', 'it', 'keep', 'latest', 'lesson', 'lik', 'link', 'load', 'look', 'lot', 'man', 'many', 'me', 'meet', 'method', 'missubmit', 'most', 'much', 'my', "n't", 'nee', 'new', 'no', 'not', 'obtain', 'of', 'on', 'or', 'ord', 'pag', 'pap', 

In [20]:
training = []
output = []

output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)


# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])


In [21]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [22]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

    # Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    # fitting and saving the model
hist=model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5',hist)

print("model created")


Epoch 1/200
39/39 [==============================] - 0s 752us/step - loss: 2.0491 - accuracy: 0.1892
Epoch 2/200
39/39 [==============================] - 0s 862us/step - loss: 1.6854 - accuracy: 0.3768
Epoch 3/200
39/39 [==============================] - 0s 766us/step - loss: 1.2982 - accuracy: 0.5967
Epoch 4/200
39/39 [==============================] - 0s 903us/step - loss: 0.9996 - accuracy: 0.6077
Epoch 5/200
39/39 [==============================] - 0s 876us/step - loss: 0.6923 - accuracy: 0.7373
Epoch 6/200
39/39 [==============================] - 0s 804us/step - loss: 0.6133 - accuracy: 0.7712
Epoch 7/200
39/39 [==============================] - 0s 734us/step - loss: 0.4767 - accuracy: 0.8184
Epoch 8/200
39/39 [==============================] - 0s 774us/step - loss: 0.3901 - accuracy: 0.8871
Epoch 9/200
39/39 [==============================] - 0s 770us/step - loss: 0.3929 - accuracy: 0.8896
Epoch 10/200
39/39 [==============================] - 0s 802us/step - loss: 0.2916 - accura

In [23]:
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [24]:
from tensorflow.keras.models import load_model
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

with open('intents.json') as json_data:
    intents = json.load(json_data)

model=load_model('chatbot_model.h5')

Loading Pickle.....


In [33]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    results = model.predict(np.array([bow(sentence, words)]))[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) 
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    return random.choice(i['responses'])

            results.pop(0)
            

ERROR_THRESHOLD = 0.25


In [36]:
flag= True
while (flag==True) :
  input_data=input("You: ")
  input_data= input_data.lower()
  if input_data not in ['bye','goodbye','quit']:
    answer = response(input_data)
    print("Bot:", answer)
  else:
    flag=False
    print("Bot: Bye! take care")



You: Hey
Bot: how can we help you today
You: Trouble uploading homework
Bot: Please verify your Internet connection and retry.
You: How to get this app?
Bot: Please use this link to get skedu in computer or iphone, **http://www.jpnme.com/Common**
You: How will I win skedu badges?
Bot: For getting badges, you should do the activities regularly and try to perform well
You: Facing issues watching videos
Bot: please check your internet connection and try again
You: Any new updates for this app?
Bot: A new update to our SkEdu application is available in your playstore. If updation not available in your playstore please uninstall old SkEdu application and install our new version.
You: Thankyou
Bot: Have a great day!
You: Bye
Bot: Bye! take care
